## TensorFlow会话与神经网络实现

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》

### 3.3 TensorFlow运行模型——会话

使用 Session 来执行定义好的运算，Session 拥有并管理 Tensorflow 程序运行时的所有资源，所有的运算都要通过Session执行。当所有计算完成之后需要关闭会话来帮助系统回收资源，否则就可能出现资源泄露的问题。

In [3]:
#第一种模式
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")

result = a + b
sess = tf.Session()
print(sess.run(result))
sess.close() #关闭会话使运行中使用到的资源释放

[3. 5.]


若程序因异常而退出，Session.close() 函数可能就不会被执行到，从而导致资源泄露。  
解决这个问题：通过 Python 的上下文管理器 with 来使用 Session

In [5]:
#上下文管理器with模式
with tf.Session() as sess:
    print(sess.run(result))
# 不需要调用 sess.close() 函数

[3. 5.]


Tensorflow不会自动生成默认的Session ，需要手动指定。  

    tf.Tensor.eval() 计算张量的取值  
    tf.InteractiveSession() 直接构建默认对话，省去将产生的会话注册为默认会话的过程

### 3.4 TensorFlow实现神经网络

TensorFlow游乐场是一个通过网页浏览器就可以训练的简单神经网络并实现了可视化训练过程的工具：
    
    http://playground.tensorflow.org  

每一条边代表一个参数，边上的颜色体现了参数的取值，边的颜色越深，参数取值的绝对值越大；当边的颜色接近白色时，参数的取值接近于0。  
和边类似，当节点的输出值的绝对值越大时，颜色越深。黄色负，蓝色正。

    tf.matmul(A,B) 矩阵乘法  
    tf.Variable() 保存和更新神经网络中的参数，该运算的输出结果是一个张量  
    tf.random_normal([2,3],mean=0,stddev=2) 产生2*3随机正态矩阵，均值为0，标准差为2  

下面通过变量实现神经网络的参数并实现前向传播。  
注意：一个变量的值在被使用之前，这个变量的初始化过程需要被明确地调用，因为虽然在变量定义时给出了变量初始化的方法，但这个方法并没有被真正运行，需要通过w1.initializer和w2.initializer来给变量赋值。

In [9]:
# 通过变量实现神经网络的参数并实现前向传播
import tensorflow as tf

# 1. 声明 w1, w2 两个变量
# 通过seed设定随机种子，可以保证每次运行结果一样
# tf.random_normal 随机生成 2*3 大小的矩阵，服从正太分布 均值 mean,标准差 stddev = 1
# 注意矩阵的大小
w1 = tf.Variable(tf.random_normal([2,3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3,1], stddev=1, seed=1))

# 2. 暂时将输入的特征向量定义为一个常量，x 是1×2矩阵
x = tf.constant([[0.7, 0.9]])

# 3.定义前向传播算法的神经网络
a = tf.matmul(x, w1) # 得到隐藏层
y = tf.matmul(a, w2) # 得到输出层

# 4.调用会话运行计算和输出结果
with tf.Session() as sess:
    sess.run((w1.initializer,w2.initializer)) # 分别初始化 w1,w2 两个变量
    #两重括号因为input must be a dictionary
    print(sess.run(y))

#不使用with（不推荐）：
# sess = tf.Session()
# sess.run(w1.initializer)
# sess.run(w2.initializer)
# print(sess.run(y))
# sess.close()

[[3.957578]]


当变量多的时候，可以使用以下函数来实现初始化所有变量的过程：  
    
    init_op = tf.global_variables_initializer()  
    sess.run(init_op)

    tf.global_variables() 拿到当前计算图上所有的变量  
    tf.trainable_varibales() 得到所有需要优化的参数  
如果声明变量时参数trainable为True，那么这个变量将会被加入到GraphKeys.TRAINABLE_VARIABLES集合。

一个变量在构建之后，它的类型就不能再改变了；  
维度可能改变，但需要设置参数validate_shape=False（该用法实践中罕见）。

In [ ]:
# 报错类型不匹配，random_normal结果默认类型tf.float32  
# 声明变量  
w1=tf.Variable(tf.random_normal([2,3],stddev=1),name="w1")  
w2=tf.Variable(tf.random_normal([3,1],stddev=1,dtype=tf.float64,seed=1),name="w2")  
w1.assign(w2)  # tf.assign(A, new_number)函数的功能主要是把A的值变为new_number  
'''
程序将报错：  
TypeError: Input 'value' of 'Assign' Op has type float64 that does not match type float32 of argument 'ref'.
'''

反向传播算法实现迭代过程，每次迭代开始首先需要选取一小部分数据，叫做一个batch。  
placeholder机制用于提供输入数据。

In [18]:
# placeholder实现前向传播算法
w1=tf.Variable(tf.random_normal([2,3],stddev=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1))

# 定义placeholder作为存放输入数据的地方。类型需要定义，维度不一定要定义
# 如果维度是确定的，给定维度会降低出错概率
x=tf.placeholder(tf.float32,shape=(3,2),name="input") # 可以得到3个前向传播结果
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(y, feed_dict={x:[[0.7,0.9],[0.1,0.4],[0.5,0.8]]}))
    # feed_dict是一个字典，给出每个用到的placehold的取值

[[8.133026 ]
 [2.9532526]
 [6.8464265]]


下面定义简单的损失函数，通过tf定义了反向传播算法：

In [24]:
#sigmoid函数将y转换为0-1之间的数值，转换后y代表预测是正样本的概率，1-y为负样本的概率
y=tf.sigmoid(y)
#定义常用损失函数（交叉熵）来刻画预测值和真实值的差距
cross_entropy = -tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y,1e-10,1.0))
    +(1-y_)*tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
#定义学习率
learning_rate = 0.001
#定义反向传播算法来优化神经网络的参数
train_step=\
    tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

TensorFlow支持10种不同的优化器，常用的三种优化方法有： 

    tf.train.GradientDescentOptimizer  
    tf.train.AdamOptimizer  
    tf.train.MomentumOptimizer

反向传播算法train_step定义之后，通过运行
    
    sess.run(train_step)
    
就可以对所有在GraphKeys.TRAINABLE_VARIABLES集合中的变量进行优化，使得在当前batch下损失函数更小。

下面给出完整程序训练神经网络解决二分类问题：

In [42]:
import tensorflow as tf

# 通过 numpy 工具包生成模拟数据集
from numpy.random import RandomState

# 定义训练数据 batch 的大小
batch_size = 8

# 定义神经网络的参数，沿用前面的结构
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# 在 shape 的维度上使用 None 可以方便使用不大的 batch 的大小，在训练时把数据分成比较小的 batch
# 数据集比较大时，将大量数据放进一个 batch 会导致内存溢出
x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")


# 定义神经网络的前向传播过程
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
y = tf.sigmoid(y)

# 定义损失函数和反向传播算法
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
                                + (1 - y_) * tf.log(tf.clip_by_value(1-y, 1e-10, 1.0)))


train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# 通过随机数生成一个模拟数据集
rdm = RandomState(1)
data_size = 128
X = rdm.rand(data_size,2)

# 定义规则来给出样本的标签，x1 + x2 < 1 .为正样本（1），其他为负样本（0）
Y = [[int(x1+x2 < 1)] for (x1,x2) in X]

# 创建 Session 来运行 tf 程序
with tf.Session() as sess:
    # 初始化变量 
    sess.run(tf.global_variables_initializer())
    
    # 输出目前（未经训练）的参数取值。
    print(sess.run(w1))
    print(sess.run(w2))
    print("\n")
    
    # 设定训练的 轮数
    STEPS = 5000
    for i in range(STEPS):
        # 每次选取 batch_size 个样本进行训练
        start = (i*batch_size) % data_size
        end = (i*batch_size) % data_size + batch_size
        # 通过选取的样本 训练神经网络，并更新参数
        sess.run([train_step, y, y_], feed_dict={x:X[start:end], y_:Y[start:end]})
        # 每隔一段时间，计算在所有数据上的交叉熵并输出
        if i % 1000 ==0:
            """
            随着训练的进行，交叉熵是逐渐变小的，越小则越说明，
            预测值与真实值之间的差距越小
            每隔一段时间计算在所有数据上的交叉熵并输出
            """
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x:X, y_:Y})
            print("After %d training step(s), cross entropy on all data is %g" % (i, total_cross_entropy))
    print("\n")
    print(sess.run(w1))
    print(sess.run(w2))
    """
    训练完后，神经网络参数的值，w1 ,w2 的值已经发生了变化，也就是训练的结果。
    它使得这个神经网络可以更好的拟合提供的训练数据。具有更好的泛化能力。
    """

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]


After 0 training step(s), cross entropy on all data is 1.89805
After 1000 training step(s), cross entropy on all data is 0.655075
After 2000 training step(s), cross entropy on all data is 0.626172
After 3000 training step(s), cross entropy on all data is 0.615096
After 4000 training step(s), cross entropy on all data is 0.610309


[[ 0.02476983  0.56948674  1.6921941 ]
 [-2.1977348  -0.23668921  1.1143895 ]]
[[-0.45544702]
 [ 0.49110925]
 [-0.98110336]]


总结训练神经网络的过程：  
①定义神经网络的结构和前向传播的输出结果  
②定义损失函数以及选择反向传播优化的算法（常用优化算法：）
   
    tf.train.GradientDescentOptimizer
    tf.train.AdamOptimizer
    tf.train.MomentumOptimizer

③定义 tf.Session 并在训练数据上反复运行反向传播优化的算法  
无论神经网络的结构如何变化，这三个步骤是不变的。